# Remote

In [26]:
# Set working directory
domain = "fend01"
dir = "exp/initial"
partition = "sbinlab_gpu"

remotedir = f"~/OrthoIDP/{dir}"

print(f"CONTENTS IN {remotedir} AT {domain.upper()}:")
!ssh $domain "cd $remotedir && ls"

CONTENTS IN ~/OrthoIDP/exp/initial AT FEND01:
data
initial.json
results
submit.sh


In [2]:
# Check availability
!ssh $domain sinfo -p $partition

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
PARTITION   AVAIL  TIMELIMIT  NODES  STATE NODELIST
sbinlab_gpu    up   infinite      1 drain* node152
sbinlab_gpu    up   infinite      2    mix node[150-151]


In [5]:
# Check queue
!ssh $domain "squeue -u fknudsen"

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          38293963 sbinlab_g idpevo_h fknudsen  R 5-04:34:27      1 node151
        38325169_0 sbinlab_g  initial fknudsen  R    5:33:05      1 node150
        38325169_1 sbinlab_g  initial fknudsen  R    5:33:05      1 node150
        38325169_2 sbinlab_g  initial fknudsen  R    5:33:05      1 node150


In [27]:
# Check submit script
!ssh $domain "cat $remotedir/submit.sh"

#!/bin/bash
#SBATCH --job-name=initial
#SBATCH --partition=sbinlab_gpu
#SBATCH --array=0-5%6
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --gres=gpu:1
#SBATCH -t 24:00:00
#SBATCH -o results/out
#SBATCH -e results/err


# Getting job input file
input_files=($(ls data/*.fasta))
input_file=${input_files[$SLURM_ARRAY_TASK_ID]}

# Displaying job info
echo "[`date`] STARTED Job Array ID: $SLURM_ARRAY_TASK_ID | Job ID: $SLURM_JOB_ID | Input: $input_file"

# DeiC env settings
source /groups/sbinlab/fpesce/.bashrc
conda activate openmm

# Submitting simulation
python ../../src/simulate_openmm.py -f $input_file

echo "[`date`] FINISHED Job Array ID: $SLURM_ARRAY_TASK_ID | Job ID: $SLURM_JOB_ID | Input: $input_file"


In [13]:
# Submit
!ssh $domain "cd $remotedir && sbatch submit.sh"

/etc/profile.d/lang.sh: line 19: warning: setlocale: LC_CTYPE: cannot change locale (UTF-8): No such file or directory
/Users/frederikespersenknudsen/Local/OrthoIDP


In [ ]:
# Cancel job
jobid = ""
!ssh $domain scancel $jobid

In [31]:
# Copy results to BINF
!scp -r "$domain:$remotedir/results" $dir

traj.log                                      100%   14MB   6.2MB/s   00:02    
top.pdb                                       100%   11KB   2.5MB/s   00:00    
simulate.log                                  100%  583    10.9KB/s   00:00    
restart.chk                                   100%   87KB   2.8MB/s   00:00    
system.xml                                    100%   25KB   1.2MB/s   00:00    
traj.dcd                                      100%  271MB  12.1MB/s   00:22    
traj.log                                      100%   14MB   9.4MB/s   00:01    
system.xml                                    100% 9110   558.4KB/s   00:00    
restart.chk                                   100%   79KB   2.6MB/s   00:00    
simulate.log                                  100%  511    28.5KB/s   00:00    
top.pdb                                       100% 3749   260.3KB/s   00:00    
traj.dcd                                      100%   96MB  10.9MB/s   00:08    
out                                     